In [1]:
import IPython.display
import sympy as sp
from sympy.vector import CoordSys3D
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from sympy import init_printing,latex
sp.init_printing(use_latex=True)

![alt text](Task_2.png "Task_2.png")

# RO:
The system consists of two bodies: A (cylinder with a hollow tube) and M - particle
# Motion:
M - translatory motion
A - rotation

# Condition:
I want to describe the motion, where the new coordinate system is:
$\begin{bmatrix} x \\
                  y\\
                  x
  \end{bmatrix} =
  R_\phi \begin{bmatrix}
                  x_1\\
                  y_1\\
                  z_1
  \end{bmatrix}$

$$ \begin{matrix}
    initial & '2' & final\\
    y_0 = 0 & y_2 = r & y_f - ? \\
    \dot{y_0} = 0.4 & \dot{y_2} - ? & \dot{y_f} - ?\\
    t_0 = 0 & t_2 - ? & t_f - ? \end{matrix} $$
# Kinematics analysis:
$\epsilon = 0$
$$ \vec{a}_{tr} = \vec{\omega} \times (\vec{\omega} \times \vec{OM}) + \vec{\epsilon} \times \vec{OM} = \vec{\omega} \times (\vec{\omega} \times \vec{OM})$$
$$ \vec{a}_{cor} = 2 \vec{\omega} \times \vec{V}_{rel}$$

# Force analysis:
$$ma_{rel} = \sum{F} + \Phi_{tr} + \Phi_{cor}$$
There is no friction between M and A, so the $\sum{F} = \vec{N} + m\vec{g}$.
$\Phi_{tr} = -m\vec{a}_{tr}$
$\Phi_{cor} = -m\vec{a}_{cor}$

# Solution:
$$ma_{rel} = \vec{N} + m\vec{g} -m\vec{a}_{tr} + -m\vec{a}_{cor} $$


Set initial conditions and constants

In [2]:
# constants
r = 0.5
m = 0.02
omega = [[np.pi], [0], [0]]

# initial conditions
t_0 = 0
x_0 = 0
dx_0 = 0.4
phi_0 = [[0], [0], [0]]

In [1]:
# init symbols for parameters
t_s, dy_s, m_s, r_s, omega_s = sp.symbols("t, y', m, r, omega")
y = sp.symbols('y', cls=sp.Function)

NameError: name 'sp' is not defined

## Kinematics solution:

### Set the coordinates of point M and O
There are two coordinate systems:
Transport

$ M_{tr} (0, 0, x)$

Static
$ M = R_\phi M_{tr}$

In [16]:
phi_s = omega_s * t_s

print(omega_s)
M_tr = sp.Matrix([[0], [y_s], [0]])

rot_matrix = sp.Matrix([[1, 0, 0],
                        [0, sp.cos(phi_s), -sp.sin(phi_s)],
                        [0, sp.sin(phi_s), sp.cos(phi_s)]])

M = rot_matrix * M_tr
print(f'Coordinates of point M is static coordinate system: {M[:]}')

Matrix([[pi], [0], [0]])
Coordinates of point M is static coordinate system: [0, y*cos(Matrix([
[pi*t],
[   0],
[   0]])), y*sin(Matrix([
[pi*t],
[   0],
[   0]]))]


### Find Velocity of the point M:
$V = V_{rel} + V_{tr} = \dot \vec{x} + \vec{\omega} \times \vec{OM}$

In [5]:
# relative velocity of the point M in the coord system of the body A
v_rel_trans_coord_sys = sp.Matrix([[0], [dy_s], [0]])
print(f'Relative velocity in transport coord system: Vm rel= {v_rel_trans_coord_sys[:]}')

# relative velocity of the point M in static coord system
v_rel_s = rot_matrix * v_rel_trans_coord_sys
# transport velocity
OM = M
omega_s = sp.Matrix([[sp.pi], [0], [0]])
v_tr_s = omega_s.cross(OM)

# full velocity
v_s = v_tr_s + v_rel_s

print(f'Full velocity: Vm = {v_s[:]}')

Relative velocity in transport coord system: Vm rel= [0, y', 0]
Full velocity: Vm = [0, -pi*y*sin(omega*t) + y'*cos(omega*t), pi*y*cos(omega*t) + y'*sin(omega*t)]


### Find transport and coriolis acceleration of the point M:
$$ \vec{a}_{tr} =  \vec{\omega} \times (\vec{\omega} \times \vec{OM})$$
$$ \vec{a}_{cor} = 2 \vec{\omega} \times \vec{V}_{rel}$$

In [10]:
a_tr_s = omega_s.cross(omega_s.cross(OM))
a_cor_s = 2 * omega_s.cross(v_rel_s)



print(f'Coriolis acceleration: a_cor={a_cor_s[:]}')
print(f'Transport acceleration: a_tr={a_tr_s[:]}')

Coriolis acceleration: a_cor=[0, -2*pi*y'*sin(omega*t), 2*pi*y'*cos(omega*t)]
Transport acceleration: a_tr=[0, -pi**2*y*cos(omega*t), -pi**2*y*sin(omega*t)]


P:

In [11]:
g_s = sp.Matrix([[0], [0], [9.8]])
P = m * g_s
P_A_cs = rot_matrix * P
print(f'Weight of M in static coordinate system: P = {P}')
print(f'Weight of M in coordinate system of body A: P = {P_A_cs}')

Weight of M in static coordinate system: P = Matrix([[0], [0], [0.196000000000000]])
Weight of M in coordinate system of body A: P = Matrix([[0], [-0.196*sin(omega*t)], [0.196*cos(omega*t)]])


### Let's calculate the vectors and projections on axis Ox for all forces:
N:

In [12]:
P_magnitude = sp.sqrt(P[0] ** 2 + P[1] ** 2 + P[2] ** 2)

# normal reaction force in static coord system
normal_reaction_force = sp.cos(phi_s) * P_A_cs

normal_reaction_force_x = sp.Matrix([[0],[0],[P_A_cs[2]]])
print(f'Normal reaction force in static coordinate system: N = {normal_reaction_force}')
print(f'Normal reaction force in coordinate system of body A: N = {normal_reaction_force_x}')

Normal reaction force in static coordinate system: N = Matrix([[0], [-0.196*sin(omega*t)*cos(omega*t)], [0.196*cos(omega*t)**2]])
Normal reaction force in coordinate system of body A: N = Matrix([[0], [0], [0.196*cos(omega*t)]])


$$y: \ddot y = - a_{tr}_y - a_{cor} + g* sin(\phi)$$
$$y: \ddot y = y \pi^2  sin(\omega t) cos(\omega t) + g* sin(\omega t)$$



Laplace transform:
$$ L(\ddot y) = s^2 \overline{y} - s y(0) - \dot y(0)$$
$$ L(y) = \overline{y} $$

$$L(\pi^2  sin(\omega t) cos(\omega t))  = \pi^2 * \frac{\omega}{s^2 + \omega ^2} * \frac{s}{s^2 + \omega ^2}$$
$$ L(g* sin(\omega t)) = g * \frac{\omega}{s^2 + \omega ^2} $$


$$ \begin{cases} s^2 \overline{y} - \overline{y} * \pi^2 * \frac{\omega}{s^2 + \omega ^2} * \frac{s}{s^2 + \omega ^2} - sy(0) -\dot y(0) =  g * \frac{\omega}{s^2 + \omega ^2} \\
 \dot y (0) = 0.4 \\
 y(0)= 0 \end{cases}$$

$$ \overline{y} (s^2 - \pi^2 * \frac{\omega}{s^2 + \omega ^2} * \frac{s}{s^2 + \omega ^2} )- s * 0 - 0.4 = g * \frac{\omega}{s^2 + \omega ^2} $$
$$\overline{y} =\frac{ g * \frac{\omega}{s^2 + \omega ^2}  + 0.4}{(s^2 - \pi^2 * \frac{\omega}{s^2 + \omega ^2} * \frac{s}{s^2 + \omega ^2} )}$$

$$ \overline{y} = \frac{g*\omega}{s^2 (s^2 + \omega^2) -\pi^2*\omega * s } + 0.4 * \frac{s^2 + \omega^2}{s^2 * (s^2 + \omega^2) - \pi^2 * \omega * s}$$

$$ \overline{y} =

In [15]:
ddy = -a_tr_s[1] - a_cor_s[1] + P_A_cs[1]

dy = sp.integrals.integrate(ddy, t_s)
y = sp.integrals.integrate(ddy, (t_s, 0, r_s))

IPython.display.display_latex(y)